In [ ]:
import os
import time
import shutil
import arcpy
from arcpy import env
from arcgis.gis import GIS

# Authenticate with GIS (using ArcGIS Online or ArcGIS Enterprise)
gis = GIS('home')

# Get the logged-in user
user = gis.users.me

# Print basic user information
print(f"Username: {user.username}")
print(f"Full Name: {user.fullName}")
print(f"Email: {user.email}")
print(f"Role: {user.role}")

# Print user groups
print("\nUser Groups:")
user_groups = user.groups
for group in user_groups:
    print(f"Group Title: {group.title}, Group ID: {group.id}")

In [ ]:
# Path to the GDB where the images are stored
gdb_path = r"\N2401.gdb"
env.workspace = gdb_path
raster_paths = [os.path.join(gdb_path, image) for image in arcpy.ListRasters()]

# nodata_value = 0
# for raster_path in raster_paths:
#     raster = arcpy.Raster(raster_path)
#     band_count = raster.bandCount
#     nodata_values = [[i + 1, nodata_value] for i in range(band_count)]
#     arcpy.management.SetRasterProperties(in_raster=raster_path, nodata=nodata_values)

publish_layers = [(path, os.path.basename(path)) for path in raster_paths]
raster_paths


In [25]:
# # Titles corresponding to the raster paths - Do not include special characters: !@#$%^&*()
# titles = [
#     "N1903_4_21_Early_psscene_8b",
#     "N1903_5_15_Late_psscene_8b"    
# ]

# # Check if the number of titles matches the number of raster paths
# if len(raster_paths) != len(titles):
#     raise ValueError("The number of titles does not match the number of raster paths.")

In [14]:
# Define the tags and summary parameters
tags = "On-farm, N trials"
summary = ""

# Sharing level
organization = True
public = True
groups = "Strip Trials 2023"
portal_folder = None

# Set the scale
min_scale = 12
max_scale = 23

In [15]:
# Define scale dictionary
scale_dict = {
    0: 591657527.591555,
    1: 295828763.795777,
    2: 147914381.897889,
    3: 73957190.948944,
    4: 36978595.474472,
    5: 18489297.737236,
    6: 9244648.868618,
    7: 4622324.434309,
    8: 2311162.217155,
    9: 1155581.108577,
    10: 577790.554289,
    11: 288895.277144,
    12: 144447.638572,
    13: 72223.819286,
    14: 36111.909643,
    15: 18055.954822,
    16: 9027.977411,
    17: 4513.988705,
    18: 2256.994353,
    19: 1128.497176,
    20: 564.248588,
    21: 282.124294,
    22: 141.062147,
    23: 70.5310735
}

nodata_value = 0

# Define temporary cache folder and package file path
temp_cache_folder = r"C:\Temp_tile_cache_folder"
os.makedirs(temp_cache_folder, exist_ok=True)
env.workspace = temp_cache_folder
env.overwriteOutput = True

try:
    # for input_raster, title in zip(raster_paths, titles):
    for input_raster, title in publish_layers:
        is_published = gis.content.search(title)
        # is_published = gis.content.search("this is a test layer")
        if is_published:
            print(f'{title} is already published.')
        else:
            in_cache_name = "temp_tile_cache"
            temp_package_file = os.path.join(temp_cache_folder, f"{title}.tpkx")

            try:
                # Manage tile cache
                print(f"Managing tile cache for {title}...")
                arcpy.management.ManageTileCache(
                    in_cache_location=temp_cache_folder,
                    manage_mode="RECREATE_ALL_TILES",
                    in_cache_name=in_cache_name,
                    in_datasource=input_raster,
                    tiling_scheme="ARCGISONLINE_SCHEME",
                    import_tiling_scheme=None,
                    max_cell_size=None,
                    min_cached_scale=scale_dict.get(min_scale),
                    max_cached_scale=scale_dict.get(max_scale)
                )
                # print(f"Tile cache managed successfully for {title}.")

                # Export tile cache
                print(f"Exporting tile cache for {title}...")
                arcpy.management.ExportTileCache(
                    in_cache_source=os.path.join(temp_cache_folder, in_cache_name),
                    in_target_cache_folder=temp_cache_folder,
                    in_target_cache_name=os.path.basename(temp_package_file).split('.')[0],
                    export_cache_type="TILE_PACKAGE_TPKX",
                    storage_format_type="COMPACT_V2",
                )
                # print(f"Tile cache exported successfully for {title}.")

                # arcgis API for python can be used to publish and share the content
                # gis = arcgis.GIS('home')
                # gis.content.add(...)
                # Refer to the following: 
                # https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#contentmanager

                # Share the tile package
                # print(f"Sharing the tile package for {title}...")
                arcpy.management.SharePackage(
                    in_package=temp_package_file,
                    username="",
                    password="",
                    summary=summary,
                    tags=tags,
                    credits="",
                    public=public,
                    groups=groups,
                    organization=organization,
                    publish_web_layer="TRUE",
                    portal_folder=portal_folder
                )
                print(f"Tile package shared successfully for {title}.")

                time.sleep(5)

                # Get the shared tile layer item
                tile_layer_object = gis.content.search(title)[0]
                print(f"Shared Tile Layer ID for {title}: {tile_layer_object.id}")
                # tile_layer_item = gis.content.get(tile_layer_object.id)
                # print(f"Shared Tile Layer URL for {title}: {tile_layer_item.url}")

            except arcpy.ExecuteError as e:
                print(f"Geoprocessing error for {title}: {e}")
            except Exception as e:
                print(f"An error occurred for {title}: {e}")

finally:
    # Clean up temporary cache folder
    try:
        shutil.rmtree(temp_cache_folder)
        print(f"Temporary files and directories have been cleared.")
    except PermissionError as e:
        print(f"PermissionError: {e}")


N2401_3_28_100m is already published.
N2401_5_28_100m is already published.
N2401_6_24_100m is already published.
N2401_7_5_100m is already published.
Temporary files and directories have been cleared.
